In [16]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder, LabelBinarizer
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import datetime

import load_data

### One hidden layer

In [17]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = weight_variable([size_in, size_out])
        b = bias_variable([size_out])
        act = tf.matmul(input, w) + b
#         tf.summary.histogram("weights", w)
#         tf.summary.histogram("biases", b)
#         tf.summary.histogram("activations", act)
        return act
    

def train_model(data, label="tissue", learning_rate=1e-2, extra=""):
    tf.reset_default_graph()
    LOGDIR = "/tmp/tcga_{0}/".format(str(datetime.datetime.today().date()))
    N_IN = data.train.X.shape[1]
    N_OUT = data.train.y[label].shape[1]
    N_HIDDEN = int(np.mean(N_IN + N_OUT))
    
    x = tf.placeholder(tf.float32, [None, N_IN], name="x")
    y_true = tf.placeholder(tf.float32, [None, N_OUT], name="labels")
    hidden = tf.nn.relu(fc_layer(x, N_IN, N_HIDDEN), name="hidden")
    y_pred = fc_layer(hidden, N_HIDDEN, N_OUT, name="softmax")
    
    xent = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=y_true, logits=y_pred), name="xent")
    tf.summary.scalar("xent", xent)

    train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    train_accu_summ = tf.summary.scalar("train_accuracy", accuracy)
    test_accu_summ = tf.summary.scalar("test_accuracy", accuracy)

    sess = tf.Session()
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR + "pca{0}_{1}_{2}".format(extra, label, learning_rate))
    writer.add_graph(sess.graph)
    
    # training
    t0 = time.time()
    sess.run(tf.global_variables_initializer())

    for i in range(100001):
        batch_x, batch_y = data.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_x, y_true: batch_y[label]})
        if i % 5 == 0:
            [train_accuracy, train_s] = sess.run([accuracy, train_accu_summ],
                feed_dict={x: tcga.train.X, y_true: tcga.train.y[label]})
            [test_accuracy, test_s] = sess.run([accuracy, test_accu_summ],
                feed_dict={x: tcga.test.X, y_true: tcga.test.y[label]})
            writer.add_summary(train_s, i)
            writer.add_summary(test_s, i)
            if i % 1000 == 0:
                print("step", i, "training accuracy", train_accuracy, 
                      "test_accuracy", test_accuracy)                
    print("training time:", time.time() - t0)

In [18]:
for PCA_variance in [0.6, 0.7, 0.8, 0.9]:
    tcga = load_data.read_data_sets("./data/mRNA_PCA_{0}_variance_MinMaxScaled.csv".format(PCA_variance))
    for label_name in ["tissue", "gender", "tumor"]:
        for lr in [1e-2, 1e-3, 1e-4]:
            print(PCA_variance, label_name, lr)
            tcga.train.reset_epoch()
            train_model(tcga, label=label_name, learning_rate=lr, extra=PCA_variance)

0.6 tissue 0.01
step 0 training accuracy 0.0552034 test_accuracy 0.0498155
step 1000 training accuracy 0.901925 test_accuracy 0.897601
step 2000 training accuracy 0.904345 test_accuracy 0.895756
step 3000 training accuracy 0.911721 test_accuracy 0.893911
step 4000 training accuracy 0.917368 test_accuracy 0.904059
step 5000 training accuracy 0.917022 test_accuracy 0.896679
step 6000 training accuracy 0.925781 test_accuracy 0.905904
step 7000 training accuracy 0.928316 test_accuracy 0.907749
step 8000 training accuracy 0.933733 test_accuracy 0.914207
step 9000 training accuracy 0.931543 test_accuracy 0.902214
step 10000 training accuracy 0.92532 test_accuracy 0.906827
step 11000 training accuracy 0.93454 test_accuracy 0.909594
step 12000 training accuracy 0.929699 test_accuracy 0.904982
step 13000 training accuracy 0.935001 test_accuracy 0.902214
step 14000 training accuracy 0.938458 test_accuracy 0.904059
step 15000 training accuracy 0.929238 test_accuracy 0.912362
step 16000 training a

step 33000 training accuracy 0.927625 test_accuracy 0.905904
step 34000 training accuracy 0.928662 test_accuracy 0.904982
step 35000 training accuracy 0.928201 test_accuracy 0.905904
step 36000 training accuracy 0.92774 test_accuracy 0.909594
step 37000 training accuracy 0.929008 test_accuracy 0.912362
step 38000 training accuracy 0.929353 test_accuracy 0.905904
step 39000 training accuracy 0.93016 test_accuracy 0.906827
step 40000 training accuracy 0.930736 test_accuracy 0.907749
step 41000 training accuracy 0.929469 test_accuracy 0.910517
step 42000 training accuracy 0.928547 test_accuracy 0.912362
step 43000 training accuracy 0.928086 test_accuracy 0.900369
step 44000 training accuracy 0.930852 test_accuracy 0.905904
step 45000 training accuracy 0.930275 test_accuracy 0.904982
step 46000 training accuracy 0.930045 test_accuracy 0.907749
step 47000 training accuracy 0.932465 test_accuracy 0.907749
step 48000 training accuracy 0.929584 test_accuracy 0.917897
step 49000 training accura

step 66000 training accuracy 0.910914 test_accuracy 0.893911
step 67000 training accuracy 0.911375 test_accuracy 0.893911
step 68000 training accuracy 0.911144 test_accuracy 0.894834
step 69000 training accuracy 0.911605 test_accuracy 0.894834
step 70000 training accuracy 0.911605 test_accuracy 0.895756
step 71000 training accuracy 0.911721 test_accuracy 0.895756
step 72000 training accuracy 0.912182 test_accuracy 0.895756
step 73000 training accuracy 0.912066 test_accuracy 0.895756
step 74000 training accuracy 0.912182 test_accuracy 0.895756
step 75000 training accuracy 0.912297 test_accuracy 0.895756
step 76000 training accuracy 0.912643 test_accuracy 0.895756
step 77000 training accuracy 0.912527 test_accuracy 0.895756
step 78000 training accuracy 0.912988 test_accuracy 0.896679
step 79000 training accuracy 0.913104 test_accuracy 0.897601
step 80000 training accuracy 0.913104 test_accuracy 0.899446
step 81000 training accuracy 0.913334 test_accuracy 0.899446
step 82000 training accu

step 100000 training accuracy 0.760516 test_accuracy 0.738007
training time: 191.16302108764648
0.6 gender 0.001
step 0 training accuracy 0.515155 test_accuracy 0.498155
step 1000 training accuracy 0.718912 test_accuracy 0.70203
step 2000 training accuracy 0.716953 test_accuracy 0.707565
step 3000 training accuracy 0.724905 test_accuracy 0.717712
step 4000 training accuracy 0.723637 test_accuracy 0.7131
step 5000 training accuracy 0.723522 test_accuracy 0.72048
step 6000 training accuracy 0.723176 test_accuracy 0.718635
step 7000 training accuracy 0.723407 test_accuracy 0.72417
step 8000 training accuracy 0.726634 test_accuracy 0.717712
step 9000 training accuracy 0.727556 test_accuracy 0.728782
step 10000 training accuracy 0.726864 test_accuracy 0.721402
step 11000 training accuracy 0.72744 test_accuracy 0.72417
step 12000 training accuracy 0.72283 test_accuracy 0.726015
step 13000 training accuracy 0.726749 test_accuracy 0.72417
step 14000 training accuracy 0.719604 test_accuracy 0.7

step 32000 training accuracy 0.727671 test_accuracy 0.7131
step 33000 training accuracy 0.727556 test_accuracy 0.72048
step 34000 training accuracy 0.728247 test_accuracy 0.714022
step 35000 training accuracy 0.728478 test_accuracy 0.71679
step 36000 training accuracy 0.728132 test_accuracy 0.714945
step 37000 training accuracy 0.729054 test_accuracy 0.718635
step 38000 training accuracy 0.730206 test_accuracy 0.72048
step 39000 training accuracy 0.731013 test_accuracy 0.721402
step 40000 training accuracy 0.731128 test_accuracy 0.72048
step 41000 training accuracy 0.731013 test_accuracy 0.725092
step 42000 training accuracy 0.731013 test_accuracy 0.723247
step 43000 training accuracy 0.732857 test_accuracy 0.729705
step 44000 training accuracy 0.73182 test_accuracy 0.725092
step 45000 training accuracy 0.733548 test_accuracy 0.72786
step 46000 training accuracy 0.733318 test_accuracy 0.732472
step 47000 training accuracy 0.732857 test_accuracy 0.730627
step 48000 training accuracy 0.7

...

step 49000 training accuracy 1.0 test_accuracy 0.969557
step 50000 training accuracy 1.0 test_accuracy 0.968635
step 51000 training accuracy 1.0 test_accuracy 0.97048
step 52000 training accuracy 1.0 test_accuracy 0.971402
step 53000 training accuracy 1.0 test_accuracy 0.969557
step 54000 training accuracy 1.0 test_accuracy 0.971402
step 55000 training accuracy 1.0 test_accuracy 0.969557
step 56000 training accuracy 1.0 test_accuracy 0.969557
step 57000 training accuracy 1.0 test_accuracy 0.969557
step 58000 training accuracy 1.0 test_accuracy 0.971402
step 59000 training accuracy 1.0 test_accuracy 0.969557
step 60000 training accuracy 1.0 test_accuracy 0.969557
step 61000 training accuracy 1.0 test_accuracy 0.968635
step 62000 training accuracy 1.0 test_accuracy 0.968635
step 63000 training accuracy 1.0 test_accuracy 0.968635
step 64000 training accuracy 1.0 test_accuracy 0.969557
step 65000 training accuracy 1.0 test_accuracy 0.969557
step 66000 training accuracy 1.0 test_accuracy 0.

step 87000 training accuracy 0.517345 test_accuracy 0.486162
step 88000 training accuracy 0.517345 test_accuracy 0.486162
step 89000 training accuracy 0.517345 test_accuracy 0.48524
step 90000 training accuracy 0.517345 test_accuracy 0.486162
step 91000 training accuracy 0.517345 test_accuracy 0.486162
step 92000 training accuracy 0.517345 test_accuracy 0.486162
step 93000 training accuracy 0.517345 test_accuracy 0.486162
step 94000 training accuracy 0.517345 test_accuracy 0.486162
step 95000 training accuracy 0.517345 test_accuracy 0.486162
step 96000 training accuracy 0.517345 test_accuracy 0.486162
step 97000 training accuracy 0.517345 test_accuracy 0.486162
step 98000 training accuracy 0.517345 test_accuracy 0.486162
step 99000 training accuracy 0.517345 test_accuracy 0.486162
step 100000 training accuracy 0.517345 test_accuracy 0.486162
training time: 664.2918934822083
0.9 gender 0.001
step 0 training accuracy 0.482655 test_accuracy 0.513838
step 1000 training accuracy 0.979486 te

step 25000 training accuracy 1.0 test_accuracy 0.99262
step 26000 training accuracy 1.0 test_accuracy 0.994465
step 27000 training accuracy 1.0 test_accuracy 0.993542
step 28000 training accuracy 1.0 test_accuracy 0.990775
step 29000 training accuracy 1.0 test_accuracy 0.994465
step 30000 training accuracy 1.0 test_accuracy 0.994465
step 31000 training accuracy 1.0 test_accuracy 0.995387
step 32000 training accuracy 1.0 test_accuracy 0.993542
step 33000 training accuracy 1.0 test_accuracy 0.994465
step 34000 training accuracy 1.0 test_accuracy 0.994465
step 35000 training accuracy 1.0 test_accuracy 0.993542
step 36000 training accuracy 1.0 test_accuracy 0.994465
step 37000 training accuracy 1.0 test_accuracy 0.994465
step 38000 training accuracy 1.0 test_accuracy 0.995387
step 39000 training accuracy 1.0 test_accuracy 0.995387
step 40000 training accuracy 1.0 test_accuracy 0.994465
step 41000 training accuracy 1.0 test_accuracy 0.993542
step 42000 training accuracy 1.0 test_accuracy 0.

step 65000 training accuracy 0.98352 test_accuracy 0.940037
step 66000 training accuracy 0.991472 test_accuracy 0.947417
step 67000 training accuracy 0.99297 test_accuracy 0.944649
step 68000 training accuracy 0.993316 test_accuracy 0.948339
step 69000 training accuracy 0.988706 test_accuracy 0.947417
step 70000 training accuracy 0.988936 test_accuracy 0.941882
step 71000 training accuracy 0.994007 test_accuracy 0.949262
step 72000 training accuracy 0.993316 test_accuracy 0.944649
step 73000 training accuracy 0.972917 test_accuracy 0.950185
step 74000 training accuracy 0.993431 test_accuracy 0.946494
step 75000 training accuracy 0.99055 test_accuracy 0.948339
step 76000 training accuracy 0.990895 test_accuracy 0.943727
step 77000 training accuracy 0.993661 test_accuracy 0.944649
step 78000 training accuracy 0.993892 test_accuracy 0.947417
step 79000 training accuracy 0.993892 test_accuracy 0.947417
step 80000 training accuracy 0.99297 test_accuracy 0.944649
step 81000 training accuracy

step 2000 training accuracy 0.97891 test_accuracy 0.964945
step 3000 training accuracy 0.982598 test_accuracy 0.968635
step 4000 training accuracy 0.981445 test_accuracy 0.961255
step 5000 training accuracy 0.993661 test_accuracy 0.982472
step 6000 training accuracy 0.977066 test_accuracy 0.956642
step 7000 training accuracy 0.995505 test_accuracy 0.978782
step 8000 training accuracy 0.996543 test_accuracy 0.978782
step 9000 training accuracy 0.994122 test_accuracy 0.971402
step 10000 training accuracy 0.99539 test_accuracy 0.97786
step 11000 training accuracy 0.99781 test_accuracy 0.98155
step 12000 training accuracy 0.998041 test_accuracy 0.980627
step 13000 training accuracy 0.999309 test_accuracy 0.980627
step 14000 training accuracy 0.999309 test_accuracy 0.979705
step 15000 training accuracy 0.999424 test_accuracy 0.980627
step 16000 training accuracy 0.999654 test_accuracy 0.979705
step 17000 training accuracy 0.99977 test_accuracy 0.979705
step 18000 training accuracy 0.999424 